In the paper [Extreme submillimetre starburst galaxies](https://www.aanda.org/articles/aa/full_html/2018/11/aa32671-18/aa32671-18.html#S9) there are claims made about there being galaxies with an SFR of 30,000 $M_{\odot}/yr$. We believe these objects are actually due to blending of multiple objects. 

This notebook wil work on establishing the sample of objects we will examine in this paper.

Things to do:
1. Establish the file associated with Michael's paper
2. Gather all the relevant maps, psf's etc
3. Visualise Sources

In [64]:
from astropy.table import Table, vstack,hstack
#Set some color info
import seaborn as sns
import numpy as np
import pylab as plt
import xidplus
import pymoc
%matplotlib inline
cmap=sns.cubehelix_palette(8, start=.5, rot=-.75,as_cmap=True)
vmin=-0.01
vmax=0.1

There are four tables in the appendix for the candidates, each table corresponding to a type of galaxy

In [19]:
ysb=Table.read('../../../data/MRR2018_tables/Table_A2_youngSBs.csv', format='ascii',encoding='utf-8-sig')
M82_SBs=Table.read('../../../data/MRR2018_tables/Table_A2_M82_Arp220_SBs.csv', format='ascii',encoding='utf-8-sig')
QSOs = Table.read('../../../data/MRR2018_tables/Table_A2_QSOs_SBs.csv', format='ascii',encoding='utf-8-sig')
AGN = Table.read('../../../data/MRR2018_tables/Table_A2_Type_2_AGN_SBs.csv', format='ascii',encoding='utf-8-sig')

In [20]:
ysb

RA,Dec,i,S 24 (μJy),S 250 (mJy),S 350 (mJy),S 500 (mJy),Z phot,Type,Chi,n bands,Z subm,Z comb,M * log(M sol.),SFR log(M sol. yr−1)
float64,float64,str9,float64,float64,float64,float64,float64,str3,float64,int64,float64,float64,float64,float64
159.03456,58.44533,21.37,1127.8,131.6,84.2,42.2,1.44,Scd,4.8,4,1.16,1.19,11.62,3.89
35.28232,-4.149,g = 25.31,636.1,53.7,58.3,55.1,3.27,Sab,2.4,4,3.84,2.8,11.28,3.7
35.28037,-4.14839,--,--,--,--,--,2.55,--,--,--,--,--,--,3.59
160.50839,58.67179,23.49,904.6,95.7,79.5,60.5,3.81,Sab,4.1,6,2.86,3.07,12.02,3.99
160.51041,58.67371,--,--,--,--,--,1.08,--,--,--,--,--,--,3.05
36.59817,-4.56164,25.11,389.5,59.3,67.2,54.7,3.88,Sab,2.1,5,3.53,3.68,11.49,3.85
161.98271,58.07477,22.1,264.4,44.2,45.3,33.6,4.13,sb,25.4,6,3.15,3.68,11.87,3.76


In [21]:
M82_SBs

RA,Dec,i,S 24 (μJy),S 250 (mJy),S 350 (mJy),S 500 (mJy),Z phot,Type,Chi,n bands,Z subm,Z comb,M * log(M sol.),SFR log(M sol. yr−1)
float64,float64,str9,float64,float64,float64,float64,float64,str9,float64,int64,float64,float64,float64,float64
162.33324,58.10657,22.61,516.4,56.9,52.9,59.6,2.8,Scd,6.1,8,4.09,2.89,11.69,3.72
9.64405,-44.36636,23.02,987.2,91.7,79.4,48.1,2.85,Scd,1.8,4,2.43,2.72,11.89,3.7
160.9194,57.91475,22.32,682.3,116.5,102.0,65.4,3.06,Sab,20.5,6,2.56,2.8,12.5,4.12
8.81979,-42.69724,R = 22.90,706.2,87.4,74.6,53.9,3.19,Scd,6.1,5,2.72,2.98,12.06,3.79
162.38754,57.70547,g = 24.13,519.3,65.7,65.0,56.3,3.37,Scd,1.6,3,3.47,3.37,12.15,3.74
36.71948,-3.96377,25.18,322.8,43.4,42.8,32.1,3.57,Sbc,5.2,5,3.1,3.27,12.21,3.7
162.46065,58.11701,21.96,252.5,--,28.4,41.4,4.06,sb (QSO?),7.9,6,4.81,4.13,11.97,3.98
35.73369,-5.62305,23.42,422.0,73.1,60.9,42.1,4.11,Sbc,4.1,6,2.61,2.8,12.38,4.1
--,--,--,--,--,--,--,2.8,Sbc,--,--,--,--,--,3.7


In [22]:
QSOs

RA,Dec,i,S 24 (μJy),S 250 (mJy),S 350 (mJy),S 500 (mJy),Z phot,Type,Chi,n bands,Z subm,Z comb,M * log(M sol.),SFR log(M sol. yr−1)
float64,float64,str9,float64,float64,float64,float64,float64,str3,float64,int64,float64,float64,int64,float64
164.64154,58.09799,20.95,1386.3,44.9,45.4,28.0,3.37,QSO,2.1,11,2.01,3.07,--,3.71
164.64879,58.09513,--,--,--,--,--,1.11,--,--,--,--,--,--,2.74
162.77605,58.52327,22.32,184.6,54.5,53.0,41.1,3.47,QSO,3.2,8,3.19,3.37,--,3.71
161.8541,57.91928,21.11,245.8,76.1,80.3,65.3,3.49,QSO,1.3,10,3.42,3.47,--,3.86
162.20728,58.28162,21.59,205.5,56.6,57.7,40.3,3.61,QSO,8.5,10,3.14,3.37,--,3.83
161.36092,58.03157,20.67,428.6,--,37.5,41.2,3.81,QSO,5.2,11,5.05,3.9,--,3.83
162.6812,57.55606,g = 24.35,194.9,25.5,37.0,38.3,4.5,QSO,1.3,3,4.79,4.5,--,3.79


In [23]:
AGN

RA,Dec,i,S 24 (μJy),S 250 (mJy),S 350 (mJy),S 500 (mJy),Z phot,Type,Chi,n bands,Z subm,Z comb,M * log(M sol.),SFR log(M sol. yr−1)
float64,float64,str9,float64,float64,float64,float64,float64,str3,float64,int64,float64,float64,float64,float64
33.711,-4.17344,22.53,1016.3,96.2,69.0,32.5,1.47,Scd,9.3,6,1.92,1.57,11.8,3.81
162.9173,58.80596,22.03,1964.8,183.8,133.1,78.7,2.06,Sab,45.6,6,2.11,2.09,12.22,3.81
161.75087,59.01883,23.66,1329.6,75.0,61.0,40.6,2.562,Scd,1.7,7,2.53,--,11.78,3.7
35.95578,-5.08144,23.85,1002.1,71.2,77.6,59.5,3.15,Sab,7.3,5,3.32,3.27,11.91,3.8
10.23271,-44.07592,R = 21.50,487.0,75.3,68.1,45.7,3.29,sb,13.3,5,2.67,3.07,11.75,3.77
160.33716,59.40493,22.36,1225.7,--,36.8,39.0,3.29,Scd,6.7,6,4.93,3.27,12.08,3.71
36.15258,-5.1025,24.92,627.2,43.5,53.7,41.5,3.37,Sab,2.0,5,3.6,3.47,11.69,3.71
159.78395,58.55888,g = 24.41,231.6,35.3,39.9,36.7,3.72,Sdm,0.0,3,3.91,3.68,11.32,3.79
36.9684,-5.02193,24.92,610.3,40.4,47.8,39.3,3.79,Sab,7.5,6,3.66,3.79,11.79,3.94


In [85]:
# stack tables together
all_sources=vstack((ysb,M82_SBs,QSOs,AGN))
# convert to mask table
all_sources=Table(all_sources,masked=True,copy=False)
#remove masked sources
all_sources=all_sources[all_sources['RA'].mask==False]
#add empty field column
all_sources.add_column('-'*max([len(i) for i in fields]),name='field',)

In [86]:
all_sources

RA,Dec,i,S 24 (μJy),S 250 (mJy),S 350 (mJy),S 500 (mJy),Z phot,Type,Chi,n bands,Z subm,Z comb,M * log(M sol.),SFR log(M sol. yr−1),field
float64,float64,str9,float64,float64,float64,float64,float64,str9,float64,int64,float64,float64,float64,float64,str18
159.03456,58.44533,21.37,1127.8,131.6,84.2,42.2,1.44,Scd,4.8,4,1.16,1.19,11.62,3.89,------------------
35.28232,-4.149,g = 25.31,636.1,53.7,58.3,55.1,3.27,Sab,2.4,4,3.84,2.8,11.28,3.7,------------------
35.28037,-4.14839,--,--,--,--,--,2.55,--,--,--,--,--,--,3.59,------------------
160.50839,58.67179,23.49,904.6,95.7,79.5,60.5,3.81,Sab,4.1,6,2.86,3.07,12.02,3.99,------------------
160.51041,58.67371,--,--,--,--,--,1.08,--,--,--,--,--,--,3.05,------------------
36.59817,-4.56164,25.11,389.5,59.3,67.2,54.7,3.88,Sab,2.1,5,3.53,3.68,11.49,3.85,------------------
161.98271,58.07477,22.1,264.4,44.2,45.3,33.6,4.13,sb,25.4,6,3.15,3.68,11.87,3.76,------------------
162.33324,58.10657,22.61,516.4,56.9,52.9,59.6,2.8,Scd,6.1,8,4.09,2.89,11.69,3.72,------------------
9.64405,-44.36636,23.02,987.2,91.7,79.4,48.1,2.85,Scd,1.8,4,2.43,2.72,11.89,3.7,------------------


## What fields are the sources in?


In [87]:
fields=['AKARI-NEP','AKARI-SEP','Bootes','CDFS-SWIRE',
        'COSMOS','EGS','ELAIS-N1','ELAIS-N2',
        'ELAIS-S1','GAMA-09','GAMA-12','GAMA-15',
        'HATLAS-NGP','HATLAS-SGP','HDF-N','Herschel-Stripe-82',
        'Lockman-SWIRE','SA13','SPIRE-NEP','SSDF',
        'xFLS','XMM-13hr','XMM-LSS']

In [89]:
for i,f in enumerate(fields):
    MOC=pymoc.MOC()
    pymoc.io.fits.read_moc_fits(MOC,filename=
                       'http://hedam.lam.fr/HELP/dataproducts/dmu2/dmu2_field_coverages/{}_MOC.fits'.format(f))
    in_moc=xidplus.moc_routines.check_in_moc(all_sources['RA'],all_sources['Dec'],MOC)
    all_sources['field'][in_moc]=f

In [90]:
all_sources

RA,Dec,i,S 24 (μJy),S 250 (mJy),S 350 (mJy),S 500 (mJy),Z phot,Type,Chi,n bands,Z subm,Z comb,M * log(M sol.),SFR log(M sol. yr−1),field
float64,float64,str9,float64,float64,float64,float64,float64,str9,float64,int64,float64,float64,float64,float64,str18
159.03456,58.44533,21.37,1127.8,131.6,84.2,42.2,1.44,Scd,4.8,4,1.16,1.19,11.62,3.89,Lockman-SWIRE
35.28232,-4.149,g = 25.31,636.1,53.7,58.3,55.1,3.27,Sab,2.4,4,3.84,2.8,11.28,3.7,XMM-LSS
35.28037,-4.14839,--,--,--,--,--,2.55,--,--,--,--,--,--,3.59,XMM-LSS
160.50839,58.67179,23.49,904.6,95.7,79.5,60.5,3.81,Sab,4.1,6,2.86,3.07,12.02,3.99,Lockman-SWIRE
160.51041,58.67371,--,--,--,--,--,1.08,--,--,--,--,--,--,3.05,Lockman-SWIRE
36.59817,-4.56164,25.11,389.5,59.3,67.2,54.7,3.88,Sab,2.1,5,3.53,3.68,11.49,3.85,XMM-LSS
161.98271,58.07477,22.1,264.4,44.2,45.3,33.6,4.13,sb,25.4,6,3.15,3.68,11.87,3.76,Lockman-SWIRE
162.33324,58.10657,22.61,516.4,56.9,52.9,59.6,2.8,Scd,6.1,8,4.09,2.89,11.69,3.72,Lockman-SWIRE
9.64405,-44.36636,23.02,987.2,91.7,79.4,48.1,2.85,Scd,1.8,4,2.43,2.72,11.89,3.7,ELAIS-S1


In [105]:
[all_sources[all_sources['field']==f].write('../../../data/MRR2018_tables/'+f+'_sources.csv') for f in np.unique(all_sources['field'])]

[None, None, None]

In [106]:
ls ../../../data/MRR2018_tables/

ELAIS-S1_sources.csv         Table_A2_Type_2_AGN_SBs.csv
Lockman-SWIRE_sources.csv    Table_A2_youngSBs.csv
Table_A2_M82_Arp220_SBs.csv  XMM-LSS_sources.csv
Table_A2_QSOs_SBs.csv
